This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run

Dependencies:

    pymatgen(latest development version from github)
    Polymers
    packmol

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pprint import pprint

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.packmol import PackmolRunner
from polymers.utils import get_topology

Set the start, bulk, and terminal molecules. Also set the forcefield species name mapping

In [2]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
s_charges = [-0.1187, 0.0861, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_start.add_site_property("charge", s_charges)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
b_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_bulk.add_site_property("charge", b_charges)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
e_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.1187, 0.0861, 0.0861, 0.0861]
peo_end.add_site_property("charge", e_charges)
e_head = 0
e_tail = 4

# Create polymer via random walk

In [3]:
n_units = 25
link_distance = 1.5075

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

# linear chain
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                             peo_bulk, head, tail, 
                             peo_end, e_head, e_tail, 
                             n_units, link_distance, linear_chain=True)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")
#peo_polymer_linear.molecule.to(filename="polymer_linear.xyz", fmt="xyz")

set the topology of the polymer from the linear chain

In [4]:
from pymatgen.io.lammps.topology import Topology

topology = Topology.from_molecule(peo_polymer_linear.molecule)

print len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)

pprint(topology.bonds)

177 176 325 366
[[0, 1, ('C', 'H')],
 [0, 2, ('C', 'H')],
 [0, 3, ('C', 'H')],
 [0, 4, ('C', 'O')],
 [4, 5, ('O', 'C')],
 [5, 6, ('C', 'H')],
 [5, 7, ('C', 'H')],
 [5, 8, ('C', 'C')],
 [8, 9, ('C', 'H')],
 [8, 10, ('C', 'H')],
 [8, 11, ('C', 'O')],
 [11, 12, ('O', 'C')],
 [12, 13, ('C', 'H')],
 [12, 14, ('C', 'H')],
 [12, 15, ('C', 'C')],
 [15, 16, ('C', 'H')],
 [15, 17, ('C', 'H')],
 [15, 18, ('C', 'O')],
 [18, 19, ('O', 'C')],
 [19, 20, ('C', 'H')],
 [19, 21, ('C', 'H')],
 [19, 22, ('C', 'C')],
 [22, 23, ('C', 'H')],
 [22, 24, ('C', 'H')],
 [22, 25, ('C', 'O')],
 [25, 26, ('O', 'C')],
 [26, 27, ('C', 'H')],
 [26, 28, ('C', 'H')],
 [26, 29, ('C', 'C')],
 [29, 30, ('C', 'H')],
 [29, 31, ('C', 'H')],
 [29, 32, ('C', 'O')],
 [32, 33, ('O', 'C')],
 [33, 34, ('C', 'H')],
 [33, 35, ('C', 'H')],
 [33, 36, ('C', 'C')],
 [36, 37, ('C', 'H')],
 [36, 38, ('C', 'H')],
 [36, 39, ('C', 'O')],
 [39, 40, ('O', 'C')],
 [40, 41, ('C', 'H')],
 [40, 42, ('C', 'H')],
 [40, 43, ('C', 'C')],
 [43, 44, ('C',

# Generate polymer matrix

In [5]:
#Polymer matrix settings

# constituent molecules
molecules = [peo_polymer.molecule]*4
#radius = 100.0
matrix_config = [{"number": 1, "inside box":[0,0,0,150,150,150]},
                  {"number": 1, "inside box":[0,0,0,150,150,150]},
                  {"number": 1, "inside box":[0,0,0,150,150,150]},
                  {"number": 1, "inside box":[0,0,0,150,150,150]},]
print matrix_config
len(molecules)

[{'number': 1, 'inside box': [0, 0, 0, 150, 150, 150]}, {'number': 1, 'inside box': [0, 0, 0, 150, 150, 150]}, {'number': 1, 'inside box': [0, 0, 0, 150, 150, 150]}, {'number': 1, 'inside box': [0, 0, 0, 150, 150, 150]}]


4

Use Packmol to pack the polymer chains into a box

In [6]:
pmr = PackmolRunner(molecules,
                    matrix_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    output_file="poly_packed.xyz")
packed_polymer = pmr.run()
print len(packed_polymer)
print len(peo_polymer.molecule)

packed molecule written to poly_packed.xyz
708
177


Read in the force field parameters

In [7]:
from pymatgen.io.lammps.force_field import ForceField

forcefield = ForceField.from_file("ff_data.yaml")
pprint(forcefield.bonds)

{(u'C', u'C'): [1000, 1.5075],
 (u'C', u'H'): [1000, 1.1041],
 (u'C', u'O'): [1000, 1.4115]}


# Generate lammps data file from the topology and the forcefield

In [8]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols_number = [mol_config["number"] for mol_config in matrix_config]
print mols_number
#length = radius*1.1
box_size = [[0.0, 150], 
            [0.0, 150], 
            [0.0, 150]]
# list of molecular topologies for each constituent molecules
topologies = [topology]*len(molecules)

lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(molecules, mols_number, 
                                                                   box_size, packed_polymer, 
                                                                   forcefield, topologies)


[1, 1, 1, 1]


KeyError: 354

Write the data file(uncomment the write line)

In [ ]:
print str(lammps_ff_data)

#write the data file
lammps_ff_data.write_data_file("lammps_data.dat")